In [1]:
import pandas as pd
import numpy as np
import sys

from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.
C:\Users\1019l\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\1019l\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\1019l\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\1019l\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
np.set_printoptions(threshold=sys.maxsize)

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

In [4]:
dataframe = pd.read_csv("transformed_data.csv", low_memory=False)

In [5]:
dataframe.head(5)

,Unnamed: 0,hasHeating,hasCooling,hasFireplace,furnished,hasAttachedGarage,hasGarage,hasOpenParking,hasCarport,hasPrivatePool,hasSpa,hasView,hasWaterfrontView,canRaiseHorses,hasAttachedProperty,hasLandLease,isNewConstruction,hasHomeWarranty,hasPetsAllowed,hasRentControl,isSeniorCommunity,price,imageId,imageIdCount,flooring_Carpet,flooring_Ceramic,flooring_Concrete,flooring_Hardwood,flooring_Laminate,flooring_Linoleum / Vinyl,flooring_Mixed,flooring_Other,flooring_Slate,flooring_Softwood,flooring_Tile,flooring_carpet,flooring_hardwood,flooring_laminate,flooring_tile,heating_Baseboard,heating_Forced Air,heating_Forced air,heating_Geothermal,heating_Heat pump,heating_Other,heating_Radiant,heating_Stove,heating_Wall,heating_baseboard,heating_forced air,heating_heat pump,cooling_C,cooling_Central,cooling_Evaporative,cooling_Geothermal,cooling_None,cooling_Other,cooling_Refrigeration,cooling_Solar,cooling_Wall,cooling_central,appliances_Air-To-Air Exchanger,appliances_Central Vacuum,appliances_Cooktop,appliances_Dehumidifier,appliances_Dishwasher,appliances_Disposal,appliances_Double Oven,appliances_Dryer,appliances_Dryer Included,appliances_Electric Dryer Connection,appliances_Electric Water Heater,appliances_Electronic Air Filter,appliances_Exhaust Fan/Hood,appliances_Freezer,appliances_Fuel Tank - Owned,appliances_Fuel Tank - Rented,appliances_Furnace Humidifier,appliances_Garbage disposal,appliances_Gas Dryer Connections,appliances_Gas Water Heater,appliances_Hood,appliances_Indoor Grill,appliances_Iron Filter,appliances_Microwave,appliances_None,appliances_Other,appliances_Oven,appliances_Range,appliances_Range / Oven,appliances_Range/Stove,appliances_Refrigerator,appliances_Tankless Water Heater,appliances_Trash Compactor,appliances_Trash compactor,appliances_Wall Oven,appliances_Washer,appliances_Washer Included,appliances_Water Filtration System,appliances_Water Osmosis System,appliances_Water Softener - Owned,appliances_Water Softener - Rented,appliances_dishwasher,appliances_dryer,appliances_range oven,appliances_refrigerator,appliances_washer,parkingFeatures_Attached Garage,parkingFeatures_Carport,parkingFeatures_Covered Parking,parkingFeatures_Detached Garage,parkingFeatures_Garage,parkingFeatures_None,parkingFeatures_Off street,parkingFeatures_On street,parkingFeatures_garage attached,parkingFeatures_garage detached,exteriorFeatures_Brick,exteriorFeatures_Cement concrete,exteriorFeatures_Composition,exteriorFeatures_Metal,exteriorFeatures_Other,exteriorFeatures_Shingle,exteriorFeatures_Stone,exteriorFeatures_Stucco,exteriorFeatures_Vinyl,exteriorFeatures_Wood,exteriorFeatures_Wood products,exteriorFeatures_deck,exteriorFeatures_fenced yard,exteriorFeatures_lawn,exteriorFeatures_parking,exteriorFeatures_patio,exteriorFeatures_porch,foundationDetails_Concrete,foundationDetails_Concrete Block,foundationDetails_Crawl/Raised,foundationDetails_Masonry,foundationDetails_Other,foundationDetails_Piers,foundationDetails_Retaining Wall,foundationDetails_Slab,foundationDetails_Stone,foundationDetails_Wood,view_Bay,view_City,view_City Lights,view_Inland Lake,view_Lake Superior,view_Limited,view_Mountain,view_Panoramic,view_Park,view_River,view_Territorial,view_Typical,view_Valley,view_Water,homeType_homeType_Apartment,homeType_homeType_Condo,homeType_homeType_Cooperative,homeType_homeType_Miscellaneous,homeType_homeType_Mobile_/_Manufactured,homeType_homeType_Multi_Family,homeType_homeType_Multiple_Occupancy,homeType_homeType_Single_Family,homeType_homeType_Townhouse,homeType_homeType_Vacant_Land,roofType_roofType_Asphalt,roofType_roofType_Built_up,roofType_roofType_Composition,roofType_roofType_Metal,roofType_roofType_Other,roofType_roofType_Shake/Shingle,roofType_roofType_Slate,roofType_roofType_Tile,structureType_structureType_Bungalow,structureType_structureType_Cape_cod,structureType_structureType_Colonial,structureType_structureType_Contemporary,structureType_structureType_Craftsman,structureType_structureType_French,structure

In [6]:
vgg = VGG16(weights='imagenet', include_top=False, input_shape = (128, 128, 3)) #Load pretrained model
for layer in vgg.layers:
    layer.trainable = False
    
top_model = Sequential() 
top_model.add(Flatten(input_shape=vgg.output_shape[1:]))
vgg_model = Model(inputs=vgg.input, outputs=top_model(vgg.output))

In [12]:
def process_pic(name):
    try:
        img = image.load_img(name, target_size=(128, 128))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
    except:
        img = image.load_img("../images/black.jpg", target_size=(128,128))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
    return x[0]

In [24]:
def process_all_data():
    processed = []
    for index, row in dataframe.iterrows():
        np_arr = row.drop(['imageId', 'imageIdCount', 'price']).values
        vgg_arr = vgg_model.predict(np.array([process_pic("../images/" + row['imageId'] + "_0.jpg")]))[0][:256]
        price_arr = [row['price']]
        combined_arr = np.concatenate((price_arr, np_arr, vgg_arr))
        processed.append(combined_arr)
    return np.array(processed)

In [25]:
processed_data = process_all_data()

In [27]:
np.save('image_process_data.npy', processed_data)

In [26]:
print(processed_data.shape)

(31824, 471)
